# 2021-08-09 Machine coords and part coords

Note that the log files come in terms of (x,y,z,a,c) where (x,y,z) are the translation stages and (a,c) are the R_x, R_z rotations

R_x is locked to -120, 120 degrees or so, but R_z spins freely

Looking at whether we can deduce the machine coord to part coord transformation

Note that SN2 is completely translations while SN5 or SN6 should use them a bit

In [1]:
import os
import sqlite3
import re
import datetime

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scipy.signal


In [42]:
# Keeping data in nearby folder that's entered in .gitignore
os.listdir(os.path.join("..", "..", "SN2", "20200930_1500_"))
sn2_path = os.path.join("..", "..", "SN2", "20200930_1500_", "Data.dat")
# sn2_path = os.path.join("..", "..", "SN2", "20201001_1739_", "Data.dat")


os.listdir(os.path.join("..", "..", "SN5", ))
sn5_path = os.path.join("..", "..", "SN5", "20201022_1359_", "Data.dat")

def read_data(data_path):
    """
    Reads in .dat logfile
    """
    meta_data_lines = []
    with open(data_path, 'r') as a_file:
        for line in a_file:
            # Read the metadata in first, then halt at the empty line where the rest of the data begins
            if line != "\n":
                meta_data_lines.append(line)
            else:
                data_df = pd.read_csv(a_file, header=0, delimiter=' ')

    meta_data_dict = {}
    int_regex = re.compile(r"-?\d+$")
    float_regex = re.compile(r"-?\+?\d+\.?\d*e?\+?-?\d*$")
    meta_data_regex = re.compile(r"# (.*): (.*)\n")
    for line in meta_data_lines:
        match_result = meta_data_regex.match(line)
        key, value = match_result.groups()
        if int_regex.match(value):
            value = int(value)
            continue
        if float_regex.match(value):
            value = float(value)
        meta_data_dict[key] = value
    meta_data_dict["datetime"] = datetime.datetime.strptime(meta_data_dict["Date"], "%Y%m%d_%H%M_")
    data_df["t(s)"] = data_df["t"] * 1e-3
    data_df["t(min)"] = data_df["t(s)"] / 60
    
    return meta_data_dict, data_df

_, sn2_df = read_data(sn2_path)
_, sn5_df = read_data(sn5_path)


os.listdir(os.path.join("..", "..", "SN3", "20201002_1152_"))
sn3_path = os.path.join("..", "..", "SN3", "20201002_1152_", "Data.dat")
_, sn3_df = read_data(sn3_path)

os.listdir(os.path.join("..", "..", "SN6", "20201023_1426_"))
sn6_path = os.path.join("..", "..", "SN6", "20201023_1426_", "Data.dat")
_, sn6_df = read_data(sn6_path)

In [43]:
os.listdir(os.path.join("..", "..", "SN6", "20201023_1426_"))
sn6_path = os.path.join("..", "..", "SN6", "20201023_1426_", "Data.dat")
# _, sn6_df = read_data(sn3_path)

In [44]:
os.listdir(os.path.join("..", "..", "SN3", "20201002_1152_"))
sn3_path = os.path.join("..", "..", "SN2", "20201002_1152_", "Data.dat")

In [53]:
subset = sn2_df[
    (sn2_df["LaserPower"] > 0)
    & (sn2_df["t(min)"] > 1)
]
subset[["x", "y", "z", "a", "c"]].describe()

,x,y,z,a,c
count,59332.000000,59332.000000,59332.000000,59332.000000,59332.000000
mean,-0.054456,0.008962,94.992511,0.026293,-0.052956
std,28.109790,28.121615,21.672186,0.887752,2.023723
min,-45.228000,-45.116000,62.000000,0.000000,-114.031000
25%,-28.034250,-27.983000,76.400000,0.000000,0.000000
50%,-0.181500,0.020500,92.600000,0.000000,0.000000
75%,27.944500,27.983250,112.400000,0.000000,0.000000
max,44.211000,44.222000,137.600000,30.000000,0.000000


In [50]:
subset = sn3_df[
    (sn3_df["LaserPower"] > 0)
    & (sn3_df["t(min)"] > 1)
]
subset[["x", "y", "z", "a", "c"]].describe()

,x,y,z,a,c
count,57202.000000,57202.000000,57202.000000,57202.000000,57202.000000
mean,0.008373,0.062399,92.201011,0.188805,-0.171321
std,28.211267,28.215019,20.193846,3.014974,8.238236
min,-45.241000,-45.115000,62.000000,0.000000,-208.606000
25%,-28.081000,-28.010750,74.600000,0.000000,0.000000
50%,-0.027000,0.154500,89.000000,0.000000,0.000000
75%,28.101500,28.130750,108.800000,0.000000,0.000000
max,45.036000,44.936000,132.200000,330.000000,149.431000


In [51]:
subset = sn5_df[
    (sn5_df["LaserPower"] > 0)
    & (sn5_df["t(min)"] > 1)
]
subset[["x", "y", "z", "a", "c"]].describe()

,x,y,z,a,c
count,34769.000000,34769.000000,34769.000000,34769.0,34769.0
mean,-0.005724,-0.000641,111.398594,0.0,0.0
std,26.798522,26.800641,25.285472,0.0,0.0
min,-37.910000,-37.900000,68.000000,0.0,0.0
25%,-26.805000,-26.811000,89.498000,0.0,0.0
50%,-0.044000,-0.017000,111.398000,0.0,0.0
75%,26.797000,26.786000,133.299000,0.0,0.0
max,37.901000,37.900000,154.800000,0.0,0.0


In [52]:
subset = sn6_df[
    (sn6_df["LaserPower"] > 0)
    & (sn6_df["t(min)"] > 1)
]
subset[["x", "y", "z", "a", "c"]].describe()

,x,y,z,a,c
count,34769.000000,34769.000000,34769.000000,34769.0,34769.0
mean,0.012009,-0.003523,111.398994,0.0,0.0
std,26.800279,26.798877,25.285561,0.0,0.0
min,-37.910000,-37.900000,68.000000,0.0,0.0
25%,-26.762000,-26.829000,89.498000,0.0,0.0
50%,0.045000,-0.034000,111.399000,0.0,0.0
75%,26.809000,26.807000,133.296000,0.0,0.0
max,37.901000,37.900000,154.800000,0.0,0.0


In [49]:
os.listdir(os.path.join("..", "..", "SN2", ))

['20200930_1500_', '20201001_1739_']

Note: currently most data appears to be done without the rotation stages